In [1]:
import os
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import math
import scipy.stats as stats
# from scipy.optimize import curve_fit

from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

import sklearn.preprocessing as prep
import sklearn.metrics as metrics
from sklearn import model_selection

from rfpimp import dropcol_importances
from rfpimp import importances

import pickle

c:\program files\python38\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)


## Helper Functions

#### Helper functions for plotting

In [2]:
def plot_bar_diagram(ax, data, nested=False, plots_labels=None, plots_colors=None, bar_labels='', width=0.4, 
                     trend_line=False, trend_order=1, trend_color=None, 
                     title='', length_label='', tick_labels='', tick_axis_label='',
                     orient='v', legend_loc=None, spines=None):
    """
    Plots single or multiple bar diagrams with data labels on top(aside) of the bars
    :param ax: matplotlib axes
    :param data: pandas DataFrame where rows correspond to bar plots and columns correspond to data labels (columns in data)
    :param nested: boolean. If True, bar plots are plotted on top of each other. If False bar plots are plotted besides each other
    :param bar_labels: pandas DataFrame. labels over the bars. Should be same shape as data
    :param width: float. Bar width
    :param title: string. Plot title
    :param length_label: bar height/width axis label
    :param spines: dictionary of booleans. indicates to plot borders or not. Keys=['left', 'right', 'top', 'bottom'], values=True/False
    :param orient: string. Bar orientation: 'v'-vertical; 'h'-horisontal
    """
    bars = list(data.index)
    cols = list(data.columns)
    cells = data.values.tolist()

    if bar_labels is not None:
        labels = []
        for i, lab in enumerate(bars):
            if bar_labels=='':
                labels.append([f"{j:.4f}" for j in cells[i]])
            else:
                labels.append(bar_labels[i])
#             labels = ([f"{j:.4f}" for j in cells[i]] if bar_labels=='' else bar_labels[i])

    if orient == 'v':
        for i, lab in enumerate(bars):
            label = (bars[i] if plots_labels is None else plots_labels[i])
            color = ('r' if plots_colors is None else plots_colors[i])
            x = np.arange(len(cols))
            bar = ax.bar(x=x + width * (i - len(bars) / 2), height=cells[i], width=width,
                         label=label,
                         color=color,
                         align=('center' if nested else 'edge'))
            
            if trend_line:
                trend_color = (color if trend_color is None else trend_color[i])
                _x = np.arange(len(cols))
                _y = cells[i]
                _z = np.polyfit(_x, _y, trend_order)
                _p = np.poly1d(_z)
                trend = ax.plot(_x, _p(_x), color=color, linestyle='--', linewidth=2.5)
            
            if bar_labels is not None:
                autolabel(bar, ax, labels[i], oriented='v')

        ax.set_xticks(np.arange(len(cols)))
        if tick_labels=='':
            ax.set_xticklabels(cols)
        else:
            ax.set_xticklabels(tick_labels)
        if not length_label=='':
            ax.set_ylabel(length_label, fontsize=12, fontweight='bold')
        if not tick_axis_label=='':
            ax.set_xlabel(tick_axis_label, fontsize=12, fontweight='bold')
    elif orient == 'h':
        for i, lab in enumerate(bars):
            label = (bars[i] if plots_labels is None else plots_labels[i])
            y = np.arange(len(cols))
            height = width
            bar = ax.barh(y=y + height * (i - len(bars) / 2), width=cells[i], height=width,
                          label=label,
                          color=plots_colors[i],
                          align=('center' if nested else 'edge'))
            
            if trend_line:
                trend_color = (color if trend_color is None else trend_color[i])
                _x = np.arange(len(cols))
                _y = cells[i]
                _z = np.polyfit(_x, _y, trend_order)
                _p = np.poly1d(_z)
                trend = ax.plot(_x, _p(_x), color=color, linestyle='--', linewidth=2.5)
            
            if bar_labels is not None:
                autolabel(bar, ax, labels[i], oriented='v')
            if bar_labels is not None:
                autolabel(bar, ax, labels[i], oriented='h')

        ax.set_yticks(np.arange(len(cols)))
        if tick_labels=='':
            ax.set_yticklabels(cols)
        else:
            ax.set_yticklabels(tick_labels)
        if not length_label=='':
            ax.set_xlabel(length_label, fontsize=12, fontweight='bold')
        if not tick_axis_label=='':
            ax.set_ylabel(tick_axis_label, fontsize=12, fontweight='bold')
    else:
        raise ValueError('orient parameter value error. expected v or h ')

    if not title == '':
        ttl = ax.title
        ttl.set_position([.5, 1.07])
        ax.set_title(title, fontsize=14, fontweight='bold')
    if legend_loc is not None:
        ax.legend(loc=legend_loc)
    else:
        ax.legend().set_visible(False)

    if spines is None:
        spines = {'left': True,
                  'right': True,
                  'top': True,
                  'bottom': True}
    for i in list(spines.keys()):
        ax.spines[i].set_visible(spines[i])
        
    ax.grid(True, axis='y')

def autolabel(bars, ax, labels, oriented='v'):
    """Attach a text label displaying its height/width (depends of orientation)."""
    for bar, label in zip(bars, labels):
        if oriented == 'v':
            height = bar.get_height()
            ax.annotate(label,
                        xy=(bar.get_x() + bar.get_width() / 2, height),
                        xytext=(0, 2),
                        textcoords="offset points",
                        ha='center')
        else:
            width = bar.get_width()
            ax.annotate(label,
                        xy=(width, bar.get_y() + bar.get_height() / 2),
                        xytext=(1, 0),
                        textcoords="offset points",
                        ha='left',
                        va='center')

def plot_correlation_matrix(ax, data, cmap='hot', diagonal=False, square=True, annotate=False, annot_kwargs=8):
    mask = None
    if diagonal:
        mask = np.zeros_like(data, dtype=np.bool)
        mask[np.triu_indices_from(mask)] = True
    sns.heatmap(data, ax=ax,
                mask=mask,
                square=square,
                linewidths=1.5,
                cmap=cmap,
                cbar_kws={'shrink': 1, 'ticks': [-1, -.5, 0, 0.5, 1]},
                vmin=-1,
                vmax=1,
                annot=annotate,
                annot_kws={'size': annot_kwargs})
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45, horizontalalignment='right')
    
def plot_confusion_matrix(model, X, y, ax, normalize=True, title=None):
    """Plots confusion matrix
    Parameters:
    ax =  matplotlib axes
    normalyze = normalization over truth (number_predicted / total_count_of_this_class)
    """
    model.fit(X,y)
    conf_matrix = metrics.confusion_matrix(y_true=y, y_pred=model.predict(X),
                              labels=None)  # returns matrix for sorted classes (i.e 0, 1, 2,..., n)
    classes = y.unique()
    classes.sort()
    n_classes = [len(y[y == c]) for c in classes]
    if normalize == True:
        n_matrix = [[i for j in range(len(n_classes))] for i in n_classes]
        norm_matrix = conf_matrix / np.array(n_classes)
        ns_matrix = np.array([[i for j in enumerate(n_classes)] for i in n_classes])
        labels = np.array([["{0:.2%} \n {1:d} of {2:d}".format(norm_matrix[i, j], conf_matrix[i, j],
                                                               ns_matrix[i, j]) for j, v in
                            enumerate(norm_matrix[i])]
                           for i, v in enumerate(norm_matrix)])

        sns.heatmap(norm_matrix, annot=labels, fmt='', cmap='Blues')
    else:
        sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')

    ax.set_ylabel('True Labels')
    ax.set_xlabel('Predicted Labels')
    if title is None:
        title = model.__class__.__name__
    ax.set_title(title)
    ttl = ax.title
    ttl.set_position([.5, 1.07])

    # fix for mpl bug that cuts off top/bottom of seaborn viz
    b, t = ax.get_ylim()  # discover the values for bottom and top
    b += 0.5  # Add 0.5 to the bottom
    t -= 0.5  # Subtract 0.5 from the top
    ax.set_ylim(b, t)  # update the ylim(bottom, top) values
    
def plot_mae_deciles(model, X, y_true, y_bl, index, ax, log_transform=False, cumulative=False, title=None, exp=True):
    if exp==True:
        y_pred = np.exp(model.predict(X)) - 1
        y_true = np.exp(y_true.values) - 1
        y_bl = np.exp(y_bl.values) - 1
    else:
        y_pred = model.predict(X)
    y_mae_model = np.abs(y_true - y_pred)
    y_mae_naive = np.abs(y_true - np.mean(y_bl))
    y = np.stack([y_mae_model, y_mae_naive], axis=1)
    y = pd.DataFrame(y, index=index, columns=['mae_model', 'mae_naive']).sort_values(by='mae_model', ascending=True)

    y_dec = np.zeros(len(y_true))
    n = len(y) // 10
    N = 10 * n
    for i in range(0, N, n):
        if i/n==9:
            y_dec[i:] = i/n
        else:
            y_dec[i:i+n] = i/n
    y['deciles'] = y_dec
    y['deciles'] = y['deciles'].astype('int32')

    mae_model = []
    mae_naive = []
    dec_labs = y['deciles'].unique()
    ax.set_xticks(np.arange(len(dec_labs)))
    for i in dec_labs:
        if cumulative:
            mae_model.append(np.mean(y.loc[y['deciles']<=i, 'mae_model']))
            mae_naive.append(np.mean(y.loc[y['deciles']<=i, 'mae_naive']))
            ax.set_xticklabels(np.arange(10,110,10))
            ax.set_xlabel('% of population')
            ax.set_ylabel('Mean absolute error \n (cumulative mean)')
        else:
            mae_model.append(np.mean(y.loc[y['deciles']==i, 'mae_model']))
            mae_naive.append(np.mean(y.loc[y['deciles']==i, 'mae_naive']))
            ax.set_xlabel('Deciles')
            ax.set_ylabel('Mean absolute error \n (mean per decile)')
    data = np.stack([mae_model, mae_naive], axis=1)
    data = pd.DataFrame(data, columns=['mae_model', 'mae_naive'])

    _x = data.index
    _y = data['mae_model'].values
    ax.plot(_x, _y, marker='o', color='b')
    
    _y = data['mae_naive']
    ax.plot(_x, _y, marker='o', color='r')
    
    ax.legend(['Model predictions', 'Naive predictions'])
    spines = {'left': True,
              'right': False,
              'top': False,
              'bottom': True}
    for i in list(spines.keys()):
        ax.spines[i].set_visible(spines[i])
        
    if log_transform:
        ax.set_yscale('log')
    
    if title is None:
        title = model.__class__.__name__
    ax.set_title(title)
    ttl = ax.title
    ttl.set_position([.5, 1.05])
    
    ax.grid(True)
                 
    deciles = y['deciles']
    if index is not None:
        deciles.index = index
        
    return data, deciles

def plot_sale_change_deciles(model, X, y_true, y_bl, index, ax, cumulative=False, title=None, colors=None, exp=True):
    if exp==True:
        y_pred = np.exp(model.predict(X)) - 1
        y_true = np.exp(y_true.values) - 1
    else:
        y_pred = model.predict(X)
    y_bl = np.exp(y_bl.values) - 1
    
    sale_change_pred = y_pred - y_bl
    sale_change_true = y_true - y_bl
    y = np.stack([sale_change_pred, sale_change_true], axis=1)
    y = pd.DataFrame(y, index=index, columns=['sale_change_pred', 'sale_change_true']).sort_values(by='sale_change_pred', ascending=False)

    y_dec = np.zeros(len(y_true))
    n = len(y) // 10
    N = 10 * n
    for i in range(0, N, n):
        if i/n==9:
            y_dec[i:] = i/n
        else:
            y_dec[i:i+n] = i/n
    y['deciles'] = y_dec
    y['deciles'] = y['deciles'].astype('int32')

    sale_change_pred = []
    sale_change_true = []
    dec_labs = y['deciles'].unique()
    ax.set_xticks(np.arange(len(dec_labs)))
    for i in dec_labs:
        if cumulative:
            sale_change_pred.append(np.mean(y.loc[y['deciles']<=i, 'sale_change_pred']))
            sale_change_true.append(np.mean(y.loc[y['deciles']<=i, 'sale_change_true']))
            ax.set_xticklabels(np.arange(10,110,10))
            ax.set_xlabel('% of population', fontsize=12, fontweight='bold')
            ax.set_ylabel('Sales change \n (mean per % of population)', fontsize=12, fontweight='bold')
        else:
            sale_change_pred.append(np.mean(y.loc[y['deciles']==i, 'sale_change_pred']))
            sale_change_true.append(np.mean(y.loc[y['deciles']==i, 'sale_change_true']))
            ax.set_xlabel('Deciles', fontsize=12, fontweight='bold')
            ax.set_ylabel('Sales change per advisor', fontsize=12, fontweight='bold')
    data = np.stack([sale_change_pred, sale_change_true], axis=1)
    data = pd.DataFrame(data, columns=['sale_change_pred', 'sale_change_true'])

    _x = data.index
    _y = data['sale_change_pred'].values
    ax.plot(_x, _y, marker='o', color=('tab:blue' if colors is None else colors[0]))
    
    _y = data['sale_change_true']
    ax.plot(_x, _y, marker='o', color=('tab:orange' if colors is None else colors[0]))
    
    ax.legend(['Predicted sales change', 'True sales change'])
    spines = {'left': True,
              'right': False,
              'top': False,
              'bottom': True}
    for i in list(spines.keys()):
        ax.spines[i].set_visible(spines[i])
    
    if title is None:
        title = model.__class__.__name__
    ax.set_title(title, fontsize=14, fontweight='bold')
    ttl = ax.title
    ttl.set_position([.5, 1.05])
    
    ax.grid(True)
                 
    deciles = y['deciles']
    if index is not None:
        deciles.index = index
        
    return data, deciles

def plot_sales_deciles(model, X, y_true, index, ax, cumulative=False, log_transform=False, title=None, colors=None, exp=True):
    if exp==True:
        y_pred = np.exp(model.predict(X)) - 1
        y_true = np.exp(y_true.values) - 1
    else:
        y_pred = model.predict(X)
    y = np.stack([y_pred, y_true], axis=1)
    y = pd.DataFrame(y, index=index, columns=['sales_pred', 'sales_true']).sort_values(by='sales_pred', ascending=False)

    y_dec = np.zeros(len(y_true))
    n = len(y) // 10
    N = 10 * n
    for i in range(0, N, n):
        if i/n==9:
            y_dec[i:] = i/n
        else:
            y_dec[i:i+n] = i/n
    y['deciles'] = y_dec
    y['deciles'] = y['deciles'].astype('int32')

    sales_pred = []
    sales_true = []
    dec_labs = y['deciles'].unique()
    ax.set_xticks(np.arange(len(dec_labs)))
    for i in dec_labs:
        if cumulative:
            sales_pred.append(np.mean(y.loc[y['deciles']<=i, 'sales_pred']))
            sales_true.append(np.mean(y.loc[y['deciles']<=i, 'sales_true']))
            ax.set_xticklabels(np.arange(10,110,10))
            ax.set_xlabel('% of population', fontsize=12, fontweight='bold')
            ax.set_ylabel('Sales \n (mean per % of population)', fontsize=12, fontweight='bold')
        else:
            sales_pred.append(np.mean(y.loc[y['deciles']==i, 'sales_pred']))
            sales_true.append(np.mean(y.loc[y['deciles']==i, 'sales_true']))
            ax.set_xlabel('Deciles', fontsize=12, fontweight='bold')
            ax.set_ylabel('Sales per advisor', fontsize=12, fontweight='bold')
    data = np.stack([sales_pred, sales_true], axis=1)
    data = pd.DataFrame(data, columns=['sales_pred', 'sales_true'])

    _x = data.index
    _y = data['sales_pred'].values
    ax.plot(_x, _y, marker='o', color=('tab:blue' if colors is None else colors[0]))
    
    _y = data['sales_true']
    ax.plot(_x, _y, marker='o', color=('tab:orange' if colors is None else colors[0]))
    
    if log_transform:
        ax.set_yscale('log')
    
    ax.legend(['Predicted sales', 'True sales'])
    spines = {'left': True,
              'right': False,
              'top': False,
              'bottom': True}
    for i in list(spines.keys()):
        ax.spines[i].set_visible(spines[i])
    
    if title is None:
        title = model.__class__.__name__
    ax.set_title(title, fontsize=14, fontweight='bold')
    ttl = ax.title
    ttl.set_position([.5, 1.05])
    
    ax.grid(True)
                 
    deciles = y['deciles']
    if index is not None:
        deciles.index = index
        
    return data, deciles

def plot_deciles_data(data, figname='', color='dimgrey', plot_type='bar', trend_line=False, trend_order=1, trend_color='dimgrey', cumulative_pop=False, data_labels=False, x_label=None, y_label=None):
    grd = list(data.columns)
    grd_n = len(grd)
    grd_cols_n = 2
    grd_rows_n = int(math.ceil(grd_n / grd_cols_n))

    plt_h = 4
    plt_w = 12
    
    fig_ttl = plt.figure()
    fig_ttl.set_size_inches(plt_w * grd_cols_n, 0.3)
    ax0 = fig_ttl.add_subplot()
    ax0.text(0.5, 0.5, figname, fontsize=18, horizontalalignment='center', verticalalignment='center')
    ax0.axis('off')

    fig = plt.figure()
    fig.set_size_inches(plt_w * grd_cols_n, plt_h * grd_rows_n)
    gs = gridspec.GridSpec(grd_rows_n, grd_cols_n, figure=fig, wspace=0.3, hspace=0.5)

    for idx, col in enumerate(list(data.columns)):
        if data_labels:
            labs = ['{0:.2f}'.format(i) for i in data[col]]
            bar_labels=[labs]
        else:
            bar_labels=None
        
        if cumulative_pop:
            tick_labels = np.arange(10,110,10)
        else:
            tick_labels = list(data.index)
        
        if x_label is not None:
            tick_axis_label = x_label
        else:
            tick_axis_label = ''
        if y_label is not None:
            length_label = y_label
        else:
            length_label = col
        
        if plot_type=='bar':
            plot_bar_diagram(ax=fig.add_subplot(gs[idx]),
                             data=data[col].to_frame().T,
                             bar_labels=bar_labels,
                             tick_labels = tick_labels,
                             plots_labels=None,
                             plots_colors=[color],
                             width=0.3,
                             trend_line=trend_line, 
                             trend_order=trend_order, 
                             trend_color=[trend_color],
                             title=None,
                             tick_axis_label=tick_axis_label,
                             length_label=length_label,
                             orient='v',
                             spines={'top': False, 'right': False})
        elif plot_type=='scatter':
            ax=fig.add_subplot(gs[idx])
            _x = tick_labels
            _y = data[col].values
            ax.scatter(_x, _y, marker='o', color=color)
            z = np.polyfit(_x, _y, 2)
            p = np.poly1d(z)
            ax.plot(_x, p(_x), color=color)
            
def plot_diagram_xy(data, x, y, ax, x_title=None, y_title=None, x_labels_format='{0:.0f}', title=None, pot_size=None, tick_labelsIprefix=''):
    _data = data.copy()
    _data = _data.sort_values(by=x, ascending=False)
    _data['deciles'] = pd.cut(_data[x], 10, labels=False, duplicates='drop')
#     _dec = np.zeros(len(_data[x]))
#     n = len(_data[x]) // 10
#     N = 10 * n
#     for i in range(0, N, n):
#         if i/n==9:
#             _dec[i:] = i/n
#         else:
#             _dec[i:i+n] = i/n
#     _data['deciles'] = _dec
#     _data['deciles'] = _data['deciles'].astype('int32')

    cuts = _data['deciles'].unique()
    _x = np.zeros(len(cuts))
    _y = np.zeros(len(cuts))
    for idx, i in enumerate(cuts):
        _x[len(cuts)-1-idx] = np.mean(_data.loc[_data['deciles']==i, x])
        _y[len(cuts)-1-idx] = np.mean(_data.loc[_data['deciles']==i, y])
#     cuts = _data['deciles'].unique()
#     _x = np.zeros(len(cuts))
#     _y = np.zeros(len(cuts))
#     for idx, i in enumerate(cuts):
#         _x[len(cuts)-1-idx] = np.mean(_data.loc[_data['deciles']==i, x])
#         _y[len(cuts)-1-idx] = np.mean(_data.loc[_data['deciles']==i, y])
    plot_bar_diagram(ax=ax,
                     data=pd.Series(_y).to_frame().T,
                     tick_labels=[tick_labelsIprefix + x_labels_format.format(i) for i in _x],
                     bar_labels=None,
                     orient='v',
                     width=0.3,
                     title=('{0} vs {1}'.format(y,x) if title is None else title),
                     length_label=(y if y_title is None else y_title),
                     tick_axis_label=(x if x_title is None else x_title),
                     plots_colors=['tab:blue'],
                     spines={'top': False, 'right': False})
    
def plot_diagram_deciles(data, y, ax, x_title=None, y_title=None, title=None, pot_size=None):
    _data = data.copy()
    dec_labs = _data['deciles'].unique()
    dec_mean = []
    for i in dec_labs:
        dec_samples = _data.loc[_data['deciles']==i]
        n = len(dec_samples)
        val_mean = np.mean(dec_samples[y])
        dec_mean.append(val_mean)
        
    plot_bar_diagram(ax=ax,
                     data=pd.Series(dec_mean).to_frame().T,
                     tick_labels=[i for i in dec_labs],
                     bar_labels=None,
                     orient='v',
                     width=0.3,
                     title=('{0} vs {1}'.format(y,x) if title is None else title),
                     length_label=(y if y_title is None else y_title),
                     tick_axis_label=('Deciles' if x_title is None else x_title),
                     plots_colors=['tab:blue'],
                     spines={'top': False, 'right': False})
    
def get_lift_chart(model, data, X, by_class=1, exp=True):
    _data = data.copy()
    if exp==True:
        y_pred = np.exp(model.predict(X)) - 1
#         y_true = np.exp(y_true.values) - 1
    else:
        y_pred = model.predict(X)
#     y = np.stack([y_true, y_pred], axis=1)
#     y = pd.DataFrame(y, columns=['true_y', 'predicted_y']).sort_values(by='predicted_y', ascending=False)
    _data['predicted_y'] = y_pred.reshape(-1,1)
#     y = pd.DataFrame(y_pred.reshape(-1,1), columns=['predicted_y']).sort_values(by='predicted_y', ascending=False)
    y = _data.sort_values(by='predicted_y', ascending=False)
    y_dec = np.zeros(len(y))
    n = len(y) // 10
    N = 10 * n
    for i in range(0, N, n):
        if i/n==9:
            y_dec[i:] = i/n
        else:
            y_dec[i:i+n] = i/n
    y['deciles'] = y_dec
    y['deciles'] = y['deciles'].astype('int32')

    def val_cum(arr_n, arr_val, pos):
        arr = []
        for i in range(pos + 1):
#             print(arr_val[i], arr_n[i])
            arr.append(arr_val[i] * arr_n[i])
        return np.sum(arr)

    dec_labs = y['deciles'].unique()
    total_mean = np.mean(y['predicted_y'])
#     total_mean = np.sum(y['true_y']) / len(y)
    n_samples = []
    dec_mean = []
#     dec_val_per_fa = []
    dec_lift_ovr_mean = []
    n_cum_samples = []
    cum_mean = []
#     cum_val_per_fa = []
    cum_lift_ovr_mean = []
    for i in dec_labs:
        dec_samples = y.loc[y['deciles']==i]

        n = len(dec_samples)
        val_mean = np.mean(dec_samples['predicted_y'])
#         val_sum = np.sum(dec_samples['predicted_y'])
#         val_per_fa = val_sum / n
        lift_ovr = val_mean / total_mean - 1
#         lift_ovr = val_per_fa / total_mean - 1

        n_samples.append(n)
        dec_mean.append(val_mean)
#         dec_val_per_fa.append(val_per_fa)
        dec_lift_ovr_mean.append(lift_ovr)

        n_cum = len(y.loc[y['deciles']<=i])
#         mean_cum = (val_mean if i==0 else val_cum(n_samples, dec_mean, i) / n_cum)
        mean_cum = np.sum(dec_mean)
#         val_per_fa_cum = np.sum(dec_val_per_fa)
        lift_ovr_cum = mean_cum / total_mean - 1
#         lift_ovr_cum = val_per_fa_cum / total_mean - 1

        n_cum_samples.append(n_cum)
        cum_mean.append(mean_cum)
#         cum_val_per_fa.append(val_per_fa_cum)
#         cum_lift_ovr_mean.append(lift_ovr_cum)
        cum_lift_ovr_mean.append(np.sum(dec_lift_ovr_mean))
    lift = np.stack([n_samples, dec_mean, dec_lift_ovr_mean, n_cum_samples, cum_mean, cum_lift_ovr_mean], axis=1)
    return y, total_mean, pd.DataFrame(lift, columns=['n_samples', 'dec_mean', 'dec_lift_ovr_mean', 'n_cum_samples', 'cum_mean', 'cum_lift_ovr_mean'])
#     lift = np.stack([n_samples, dec_val_per_fa, dec_lift_ovr_mean, n_cum_samples, cum_val_per_fa, cum_lift_ovr_mean], axis=1)
#     return total_mean, pd.DataFrame(lift, columns=['n_samples', 'dec_val_per_fa', 'dec_lift_ovr_mean', 'n_cum_samples', 'cum_val_per_fa', 'cum_lift_ovr_mean'])

#### Helper functions for model training

In [3]:
# Tune parameters
def param_tune(model, X_train, y_train, param_ranges, scoring, cv, refit=''):
    if refit == '': refit = list(scoring.keys())[0]
    gsearch = model_selection.GridSearchCV(estimator=model, param_grid=param_ranges, scoring=scoring, cv=cv, refit=refit, return_train_score=True,
                           n_jobs=-1)
    gsearch.fit(X_train, y_train)
    cv_res = pd.DataFrame(gsearch.cv_results_)
    cv_res_sel = []
    for i in list(scoring.keys()):
        df = cv_res.loc[cv_res['rank_test_%s' % i] == 1].head(1)
        cv_res_sel.append(df[['mean_train_%s' % i, 'std_train_%s' % i, 'mean_test_%s' % i, 'std_test_%s' % i]].values[0])

    cv_res_sel = pd.DataFrame(cv_res_sel, index=list(scoring.keys()), columns=['mean_train', 'std_train', 'mean_test', 'std_test'])
    cv_results = cv_res_sel.T
    cv_results = cv_results.apply(lambda i: np.absolute(i))
    return gsearch.best_estimator_, gsearch.best_params_, gsearch.best_score_, cv_results

# Cross-validation report
def cv_rep(model, X, y, cv, scoring, fit_params=None):
    cv_res = model_selection.cross_validate(model, X, y, scoring=scoring, cv=cv, return_train_score=True, fit_params=fit_params, n_jobs=-1)
    cv_res = [[np.mean(cv_res['train_%s' % i]), np.std(cv_res['train_%s' % i]), np.mean(cv_res['test_%s' % i]), np.std(cv_res['test_%s' % i])] for i in list(scoring.keys())]
    cv_results = pd.DataFrame(cv_res, index=list(scoring.keys()), columns=['mean_train', 'std_train', 'mean_test', 'std_test'])
    cv_results = cv_results.T
    cv_results = cv_results.apply(lambda i: np.absolute(i))
    return cv_results

# Tune model parameters and print CV report
def train_best_model(model, param_ranges, X_train, y_train, scoring, cv, refit):
    model, best_params, best_score, cv_res = param_tune(model, X_train, y_train, param_ranges, scoring, cv, refit=refit)
    print('CV results \n ===================================================')
    display(cv_res)
    print('\n Best parameters: ', best_params)
    return model, cv_res

# Tune model parameters and print CV report for calibrated classifier model
def train_best_model_cal(model, param_ranges, X_train, y_train, scoring, cv, refit):
    if model.__class__.__name__=='CalibratedClassifierCV':
         model_cal= model 
    else:
        model_cal = calib.CalibratedClassifierCV(base_estimator=model, method='sigmoid', cv=cv)
    model_cal, best_params, best_score, cv_res = param_tune(model_cal, X_train, y_train, param_ranges, scoring, cv, refit=refit)
    print(cv_res)
    print(best_params)
    return model, cv_res

def dropcol_imp_r2_metric(model, X_valid, y_valid, sample_weights):
    y_pred = model.predict(X_valid)
    return metrics.r2_score(y_valid, y_pred, sample_weight=sample_weights)

#### Load data

In [4]:
data_2018 = pd.read_excel(io='Data/Transaction Data.xlsx',
                     sheet_name='Transactions18',
                     header=0,
                     index_col=0,
                     verbose=1)

Reading sheet Transactions18


In [5]:
data_2019 = pd.read_excel(io='Data/Transaction Data.xlsx',
                     sheet_name='Transactions19',
                     header=0,
                     index_col=0,
                     verbose=1)

Reading sheet Transactions19


In [6]:
data_2019 = pd.read_excel(io='Data/Transaction Data.xlsx',
                     sheet_name='Transactions19',
                     header=0,
                     index_col=0,
                     verbose=1)

Reading sheet Transactions19


In [15]:
data_firm = pd.read_excel(io='Data/Firm Information-1.xlsx',
                     sheet_name='Rep summary',
                     header=0,
                     index_col=0,
                     verbose=1)

Reading sheet Rep summary


#### Drop refresh date column

In [8]:
data_2018 = data_2018.drop(columns=['refresh_date'])

In [9]:
data_2019 = data_2019.drop(columns=['refresh_date'])

#### Add target columns

In [10]:
data_2019.columns = [i + '_target' for i in list(data_2019.columns)]

In [11]:
data_2019 = data_2019.loc[data_2018.index]

In [16]:
data_firm = data_firm[['Channel', 'Sub channel', 'Firm name']].loc[data_2019.index]

In [17]:
data = pd.concat([data_2018, data_2019, data_firm], axis=1)

#### Deal with NaNs

All data is numeric and and naturaly is equal or more than zero. So will fill nans with zeros.

In [19]:
data = data.fillna(value=0)

#### Drop samples that have negative sales and positive redemptions

In [20]:
# Inspect positive variables
f_pos = ['sales_12M_target',
         'AUM', 
         'sales_curr',         
         'sales_12M',
         'aum_AC_EQUITY',
         'aum_AC_FIXED_INCOME_MUNI',
         'aum_AC_FIXED_INCOME_TAXABLE',
         'aum_AC_MULTIPLE',
         'aum_AC_PHYSICAL_COMMODITY',
         'aum_AC_REAL_ESTATE',
         'aum_AC_TARGET',
         'aum_P_529',
         'aum_P_ETF',
         'aum_P_MF',
         'aum_P_SMA',
         'aum_P_UCITS',
         'aum_P_UIT']

for i in f_pos:
    print(i, len(data.loc[data[i] < 0]))

sales_12M_target 2
AUM 4377
sales_curr 4
sales_12M 7
aum_AC_EQUITY 3871
aum_AC_FIXED_INCOME_MUNI 3276
aum_AC_FIXED_INCOME_TAXABLE 1694
aum_AC_MULTIPLE 1149
aum_AC_PHYSICAL_COMMODITY 90
aum_AC_REAL_ESTATE 3
aum_AC_TARGET 19
aum_P_529 14
aum_P_ETF 5
aum_P_MF 4564
aum_P_SMA 1151
aum_P_UCITS 15
aum_P_UIT 42


In [21]:
# Inspect negative variables
f_neg = ['redemption_curr',
'redemption_12M']

for i in f_neg:
    print(i, len(data.loc[data[i] > 0]))

redemption_curr 3
redemption_12M 4


In [22]:
data = data.loc[(data['sales_12M']>=0) & 
         (data['sales_12M_target']>=0) & 
         (data['sales_curr']>=0) & 
         (data['redemption_curr']<=0) & 
         (data['redemption_12M']<=0)]

#### Reverse redemptions sign

In [23]:
for i in f_neg:
    data[i] = data[i].apply(np.abs)

#### Add delta_sales = sales_12M_2019 - sales_12M_2018

In [24]:
data['delta_sales_12M_18_19'] = (data['sales_12M_target'] - data['sales_12M'])

#### Add net sales data (net_sales = sales - redumptions)

In [25]:
net_cols = {'net_no_of_sales_12M_1': ['no_of_sales_12M_1', 'no_of_Redemption_12M_1'], 
            'net_no_of_sales_12M_10K': ['no_of_sales_12M_10K', 'no_of_Redemption_12M_10K'], 
            'net_no_of_funds_sold_12M_1': ['no_of_funds_sold_12M_1', 'no_of_funds_redeemed_12M_1'], 
            'net_no_of_fund_sales_12M_10K': ['no_of_fund_sales_12M_10K', 'no_of_funds_Redemption_12M_10K'], 
            'net_no_of_assetclass_sold_12M_1': ['no_of_assetclass_sold_12M_1', 'no_of_assetclass_redeemed_12M_1'], 
            'net_no_of_assetclass_sales_12M_10K': ['no_of_assetclass_sales_12M_10K', 'no_of_assetclass_Redemption_12M_10K']}

In [26]:
for net_col in net_cols.keys():
    data[net_col] = data[net_cols[net_col][0]] - data[net_cols[net_col][1]]

In [27]:
data['net_sales_curr'] = data['sales_curr'] - data['redemption_curr']
data['net_sales_12M'] = data['sales_12M'] - data['redemption_12M']

#### Split aum columns into positive and negative columns

In [28]:
aum_cols = []
for i in list(data.columns):
    if 'aum' in i:
        aum_cols.append(i)

In [29]:
for aum_col in aum_cols:
    data['pos_' + aum_col] = data[aum_col].apply(lambda x: x if x >= 0 else 0)
    data['neg_' + aum_col] = data[aum_col].apply(lambda x: np.abs(x) if x < 0 else 0)

In [30]:
data['pos_AUM'] = data['AUM'].apply(lambda x: x if x >= 0 else 0)
data['neg_AUM'] = data['AUM'].apply(lambda x: np.abs(x) if x < 0 else 0)

#### Split net columns into positive and negative columns

In [31]:
for net_col in list(net_cols.keys()) + ['net_sales_curr', 'net_sales_12M']:
    data['pos_' + net_col] = data[net_col].apply(lambda x: x if x >= 0 else 0)
    data['neg_' + net_col] = data[net_col].apply(lambda x: np.abs(x) if x < 0 else 0)

In [40]:
list(data['Channel'].unique())

['National Broker-Dealer',
 'Dual',
 'Independent Dealer',
 'International Outlet',
 'Fee-Based Adviser',
 'Bank/Trust',
 'Private Client Group',
 'Discount',
 'Networker',
 'Asset Manager',
 'Low/Non Producer']

In [43]:
test_data_deciles_reg = pd.read_csv('Sales_reg_test_data_deciles.csv', index_col=0)
test_data_deciles_cls = pd.read_csv('NewFund_clc_test_data_deciles.csv', index_col=0)

In [44]:
test_data_deciles_reg[['Channel', 'Sub channel', 'Firm name']] = data.loc[test_data_deciles_reg.index][['Channel', 'Sub channel', 'Firm name']]
test_data_deciles_cls[['Channel', 'Sub channel', 'Firm name']] = data.loc[test_data_deciles_cls.index][['Channel', 'Sub channel', 'Firm name']]

In [99]:
reg_decile_1 = test_data_deciles_reg.loc[test_data_deciles_reg['deciles']==0]
cls_decile_1 = test_data_deciles_cls.loc[test_data_deciles_cls['deciles']==0]

In [105]:
def get_df_vals(data, col):
    N = len(data)
    df = []
    df_index = list(data[col].unique())
    for channel in df_index:
        df.append(len(data.loc[data[col]==channel]) / N)
    return pd.DataFrame(np.array(df).reshape(-1,1), index=df_index, columns=[col])

In [107]:
df = get_df_vals(reg_decile_1, 'Channel')
df

,Channel
Networker,0.008547
Independent Dealer,0.521368
National Broker-Dealer,0.405983
Fee-Based Adviser,0.014957
Dual,0.032051
Bank/Trust,0.012821
Private Client Group,0.002137
Discount,0.002137


In [113]:
# df1 = df.loc[df[0]>0.1]
# df2 = df.loc[df[0]<0.1]

# df3 = df1.append([df2.sum().sum()])
# df3.index = list(df1.index) + ['Others']

# df3

In [111]:
# ax = df3.plot.pie(y=0, figsize=(8, 8), legend=False)
# plt.savefig('temp.jpg')

In [121]:
df = get_df_vals(reg_decile_1, 'Firm name')
df.to_csv('temp.csv')
df

,Firm name
U.S. Bank Private Wealth Management,0.008547
"Raymond James Financial Services, Inc.",0.025641
Merrill Lynch,0.181624
"Ameriprise Financial Services, Inc.",0.085470
Morgan Stanley Wealth Management,0.143162
...,...
"PNC Bank, NA",0.002137
"MML Investors Services, LLC",0.002137
William Blair & Co.,0.002137
U.S. Bancorp Wealth Management,0.002137


In [120]:
df = get_df_vals(reg_decile_1, 'Sub channel')
# df.to_csv('temp.csv')
df

,Sub channel
USBT,0.008547
IBD,0.416667
NACS,0.549145
RIA,0.025641


In [115]:
df = get_df_vals(cls_decile_1, 'Channel')
# df.to_csv('temp.csv')
df

,Channel
Independent Dealer,0.575107
Networker,0.002146
National Broker-Dealer,0.360515
Fee-Based Adviser,0.032189
Private Client Group,0.004292
Dual,0.019313
Bank/Trust,0.006438


In [122]:
df = get_df_vals(cls_decile_1, 'Firm name')
df.to_csv('temp.csv')
df

,Firm name
LPL Financial LLC,0.064378
Commonwealth Financial Network,0.012876
U.S. Bank Private Wealth Management,0.002146
NYLife Securities LLC,0.002146
"Ameriprise Financial Services, Inc.",0.077253
...,...
D.A. Davidson & Co. Inc.,0.004292
"CUSO Financial Services, LP",0.002146
"SunTrust Investment Services, Inc.",0.004292
Park Avenue Securities LLC,0.002146


In [118]:
df = get_df_vals(cls_decile_1, 'Sub channel')
# df.to_csv('temp.csv')
df

,Sub channel
IBD,0.474249
USBT,0.002146
NACS,0.510730
RIA,0.012876
